#Задание 1


In [0]:
!pip install pymorphy2[fast]

     |████████████████████████████████| 51kB 1.7MB/s 
     |████████████████████████████████| 7.1MB 4.1MB/s 
     |████████████████████████████████| 256kB 45.6MB/s 
  Created wheel for DAWG: filename=DAWG-0.7.8-cp36-cp36m-linux_x86_64.whl size=774682 sha256=f487c811132844c303a49fd828f9ff92fca0be3842d1201d3913e4f061698cad
  Stored in directory: /root/.cache/pip/wheels/d4/88/d0/4e4abc83eb8f59a71e8dbd8ba99fd5615a3af1fac1ef7f8825
Successfully built DAWG


In [0]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
from google.colab import files
uploaded = files.upload() 

Saving corpus_hum.txt to corpus_hum.txt


In [0]:
import pandas as pd
from lxml import html
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
from sklearn.decomposition import TruncatedSVD, NMF, PCA
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.ensemble import RandomForestClassifier
import gensim
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import Counter,defaultdict
from string import punctuation
import os
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
%matplotlib inline

morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return ' '.join(words)

def tokenize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]

    return ' '.join(words)


In [0]:
data = open('corpus_hum.txt').read().splitlines()

data_norm = [normalize(text) for text in data]

In [0]:
data_norm = [text for text in data_norm if text]

In [0]:
data_norm[:5]

['абай василий васо иван 1900–2001 русский лингвист родиться 2 15 декабрь 1900 с.коби тифлисский губерния ныне грузия 1925 окончить факультет общественный наука ленинградский университет 1928 аспирантура 1928–1930 сотрудник кавказский историко-археологический институт ан ссср 1930 полвека работать яфетический институт затем институт язык мышление институт языкознание ан ссср ленинград 1950 москва доктор филологический наука 1962 профессор 1969 лауреат государственный премия ссср 1981 почётный член азиатский королевский общество великобритания ирландия 1966 член-корреспондент финно-угорский общество хельсинки 1973 умереть абай москва 18 март 2001',
 'также тема',
 'лингвистика языкознание языковедение',
 'ученик н.я.марра ранний работа находиться влияние идея который впоследствии отойти видный специалист история иранский язык свой родной осетинский язык автор фундаментальный историко-этимологический словарь осетинский язык 5-ти том 1958–1990 заниматься также мифология фольклор иранский 

In [0]:
w2v = gensim.models.Word2Vec([text.split() for text in data_norm], size=50, sg=1)


In [0]:
uploaded = files.upload()

Saving paraphraser.zip to paraphraser.zip


In [0]:
!unzip paraphraser.zip

Archive:  paraphraser.zip
  inflating: LICENSE                 
  inflating: corpus.xml              
  inflating: paraphrases.xml         


In [0]:
corpus_xml = html.fromstring(open('paraphrases.xml', 'rb').read())
texts_1 = []
texts_2 = []
classes = []

for p in corpus_xml.xpath('//paraphrase'):
    texts_1.append(p.xpath('./value[@name="text_1"]/text()')[0])
    texts_2.append(p.xpath('./value[@name="text_2"]/text()')[0])
    classes.append(p.xpath('./value[@name="class"]/text()')[0])
    
data = pd.DataFrame({'text_1':texts_1, 'text_2':texts_2, 'label':classes})

In [0]:
data['text_1_norm'] = data['text_1'].apply(normalize)
data['text_2_norm'] = data['text_2'].apply(normalize)

In [0]:
#data.head()

In [0]:
def get_embedding(text, model, dim):
    text = text.split()
    
    # чтобы не доставать одно слово несколько раз
    # сделаем счетчик, а потом векторы домножим на частоту
    words = Counter(text)
    total = len(text)
    vectors = np.zeros((len(words), dim))
    
    for i,word in enumerate(words):
        try:
            v = model[word]
            vectors[i] = v*(words[word]/total) # просто умножаем вектор на частоту
        except (KeyError, ValueError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector
        

In [0]:
dim = 50
X_text_1_w2v = np.zeros((len(data['text_1_norm']), dim))
X_text_2_w2v = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_w2v[i] = get_embedding(text, w2v, dim)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_w2v[i] = get_embedding(text, w2v, dim)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


In [0]:
X_text_w2v = np.concatenate([X_text_1_w2v, X_text_2_w2v], axis=1)

In [0]:
y = data['label'].values

In [0]:
train_X, valid_X, train_y, valid_y = train_test_split(X_text_w2v, y,random_state=1)
clf = RandomForestClassifier(n_estimators=100, max_depth=7, min_samples_leaf=15,
                             class_weight='balanced')
clf.fit(train_X, train_y)

In [0]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, X_text_w2v, y, cv=5, scoring = 'f1_macro')

In [0]:
#f1 для RandomForest
scores.mean()

0.42525305711564176

In [0]:
train_X, valid_X, train_y, valid_y = train_test_split(X_text_w2v, y,random_state=1)
clf = LogisticRegression(C=1000)
clf.fit(train_X, train_y)

In [0]:
scores = cross_val_score(clf, X_text_w2v, y, cv=5, scoring = 'f1_macro')

In [0]:
#f1 для логистической регрессии
scores.mean()

0.3771685113823586

In [0]:
uploaded = files.upload()

Saving ruscorpora_upos_skipgram_300_5_2018.zip to ruscorpora_upos_skipgram_300_5_2018.zip


In [0]:
!unzip ruscorpora_upos_skipgram_300_5_2018.zip

Archive:  ruscorpora_upos_skipgram_300_5_2018.zip
  inflating: ruscorpora_upos_skipgram_300_5_2018.vec  


In [0]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('ruscorpora_upos_skipgram_300_5_2018.vec')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
uploaded = files.upload()

Saving data_paraphraser_norm.csv to data_paraphraser_norm.csv


In [0]:
data_mystem = pd.read_csv('data_paraphraser_norm.csv')

In [0]:
data_mystem.head(5)

,label,text_1,text_2,text_1_norm,text_2_norm
0,0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,полицейский_NOUN разрешать_VERB стрелять_VERB ...,полиция_NOUN мочь_VERB разрешать_VERB стрелять...
1,0,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,право_ADV полицейский_NOUN на_ADP проникновени...,правило_NOUN внесудебный_ADJ проникновение_NOU...
2,0,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,президент_NOUN египет_NOUN вводить_VERB чрезвы...,власть_NOUN египет_NOUN угрожать_VERB вводить_...
3,-1,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,вернуться_VERB из_ADP сирия_NOUN россиянин_NOU...,самолет_NOUN мчс_NOUN вывозить_VERB россиянин_...
4,0,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,в_ADP москва_NOUN из_ADP сирия_NOUN вернуться_...,самолет_NOUN мчс_NOUN вывозить_VERB россиянин_...


In [0]:
dim = 50
X_text_1_w2v_newModel = np.zeros((len(data_mystem['text_1_norm']), dim))
X_text_2_w2v_newModel = np.zeros((len(data_mystem['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_w2v_newModel[i] = get_embedding(text, model, dim)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_w2v_newModel[i] = get_embedding(text, model, dim)

In [0]:
X_text_newModel = np.concatenate([X_text_1_w2v_newModel, X_text_2_w2v_newModel], axis=1)

In [0]:
train_X, valid_X, train_y, valid_y = train_test_split(X_text_newModel, y,random_state=1)
clf = RandomForestClassifier(n_estimators=100, max_depth=7, min_samples_leaf=15,
                             class_weight='balanced')
clf.fit(train_X, train_y)


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=7, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=15, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
scores = cross_val_score(clf, X_text_newModel, y, cv=5, scoring = 'f1_macro')

In [0]:
#f1 для RandomForest, модель русвекторес
scores.mean()

0.16923051236581818

In [0]:
train_X, valid_X, train_y, valid_y = train_test_split(X_text_newModel, y,random_state=1)
clf = LogisticRegression(C=1000)
clf.fit(train_X, train_y)


LogisticRegression(C=1000, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
scores = cross_val_score(clf, X_text_newModel, y, cv=5, scoring = 'f1_macro')

In [0]:
#f1 для логистической регрессии, модель русвекторес
scores.mean()

0.19357158327083385

#Результаты

Модель с русвектореса показала совсем уж плохие результаты - 0.17 с деревьями и 0.19 с логистической регрессией по f1-метрике. Брал маленькую (архив в 192 мб, что почти в два раза меньше использованной на семинаре). 
С другой стороны, моя модель показала результаты получше, но всё равно не очень - 0.42 с деревьями и 0.37 с логистической регрессией. В общем и целом, интересно то, что RandomForest для данной задачи подходит намного лучше, чем логистическая регрессия. 

# Задание 2



In [0]:
tfidf = TfidfVectorizer(min_df=3, max_df=0.4, max_features=1000)
tfidf.fit(pd.concat([data['text_1_norm'], data['text_2_norm']]))

SVD

In [0]:
svd = TruncatedSVD(200)

X_text_1_svd = svd.fit_transform(tfidf.transform(data['text_1_norm']))
X_text_2_svd = svd.fit_transform(tfidf.transform(data['text_2_norm']))
cos_SVD = cosine_distances(X_text_1_svd, X_text_2_svd)



In [0]:
#print(cos_SVD)

NMF

In [0]:
nmf = NMF(50)

X_text_1_nmf = nmf.fit_transform(tfidf.transform(data['text_1_norm']))
X_text_2_nmf = nmf.fit_transform(tfidf.transform(data['text_2_norm']))
cos_NMF = cosine_distances(X_text_1_nmf, X_text_2_nmf)

X_text_nmf = np.concatenate([X_text_1_nmf, X_text_2_nmf], axis=1)

W2V (x2) и fastText

In [0]:
cos_w2v = cosine_distances(X_text_1_w2v, X_text_2_w2v)
cos_newModel = cosine_distances(X_text_1_w2v_newModel, X_text_2_w2v_newModel)

In [0]:
#print(cos_w2v)

In [0]:
X_text_1_w2v = np.zeros((len(data['text_1_norm']), dim))
X_text_2_w2v = np.zeros((len(data['text_2_norm']), dim))

In [0]:
fast_text = gensim.models.FastText([text.split() for text in data_norm], size=50, 
                                   min_n=4, max_n=8) 

In [0]:
dim = 50
X_text_1_ft = np.zeros((len(data['text_1_norm']), dim))
X_text_2_ft = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_ft[i] = get_embedding(text, fast_text, dim)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_ft[i] = get_embedding(text, fast_text, dim)

In [0]:
cos_ft = cosine_distances(X_text_1_ft, X_text_2_ft)

In [0]:
X_text = np.concatenate([cos_SVD, cos_NMF, cos_w2v, cos_newModel, cos_ft], axis=1)

In [0]:
train_X, valid_X, train_y, valid_y = train_test_split(X_text, y,random_state=1)
clf = RandomForestClassifier(n_estimators=100, max_depth=7, min_samples_leaf=15,
                             class_weight='balanced')
clf.fit(train_X, train_y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=7, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=15, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
scores = cross_val_score(clf, X_text, y, cv=5, scoring = 'f1_macro')

In [0]:
scores.mean()

0.37034403676433864

In [0]:
train_X, valid_X, train_y, valid_y = train_test_split(X_text, y,random_state=1)
clf = LogisticRegression(C=1000)
clf.fit(train_X, train_y)


In [0]:
scores = cross_val_score(clf, X_text, y, cv=5, scoring = 'f1_macro')

In [0]:
scores.mean()

0.36750064618415185

#Результаты
Результаты логистической (0.367) регрессии улучшились по сравнению с русвекторовской моделью, однако они всё ещё не дотягивают до результатов векторной модели, обученной на семинарском корпусе. Качество RandomForest'а наоборот упало (0.37), практически сравнявшись с логистической регрессией. Возможно, при таком подходе (обучение на косинусных расстояниях) сглаживается разница, между регрессорами, однако качество всё равно становится хуже.  